# Stochastic growth model using EGM
test code

In [2]:
using Interpolations
using Roots

In [3]:
function interp1(xpt, ypt, x; method="linear", extrapvalue=nothing)

    """
    This function needs Interpolations and works like interp1 in MATLAB.
    If you want to input array{Float64,1} as xpt, choose "linear"

    Input
        xpt: grid
        ypt: vector that we want to interpolate
        x: evaluation point
        method: "linear" or "cubic"
        extravalue: default is nothing

    Return
        y: interpolated function
    """

    if extrapvalue == nothing
        y = zeros(x)
        idx = trues(x)
    else
        y = extrapvalue*ones(x)
        idx = (x .>= xpt[1]) .& (x .<= xpt[end])
    end

    if method == "linear"
        intf = interpolate((xpt,), ypt, Gridded(Linear()))
        y[idx] = intf[x[idx]]

    elseif method == "cubic"
        itp = interpolate(ypt, BSpline(Cubic(Natural())), OnGrid())
        intf = Interpolations.scale(itp, xpt)
        y[idx] = [intf[xi] for xi in x[idx]]
    end

    return y
end


interp1 (generic function with 1 method)

In [4]:
## Basic RBC model with full depreciation using Endogenous grid method

# Parameters are based on Aruoba and Fernandez(2014)

# Need following packages.
#using Interpolations
#using Roots
#include("interp1.jl")


function main()

    ##  1. Calibration

    α = 1/3    # Elasticity of output w.r.t. capital
    β = 0.95    # Discount factor

    # Productivity values
    zgrid = [0.9792 0.9896 1.0000 1.0106 1.0212]
    # This value is already transiformed by exp

    # Transition matrix
    Π             = [0.9727 0.0273 0.0000 0.0000 0.0000;
                     0.0041 0.9806 0.0153 0.0000 0.0000;
                     0.0000 0.0082 0.9837 0.0082 0.0000;
                     0.0000 0.0000 0.0153 0.9806 0.0041;
                     0.0000 0.0000 0.0000 0.0273 0.9727]

    # 2. Steady State

    k_ss = (α*β)^(1/(1-α))
    y_ss = k_ss^α
    c_ss =y_ss - k_ss

    println("Output = $y_ss, Capital = $k_ss, Consumption = $c_ss")

    # We generate the grid of capital in the next period
    step = 0.00001
    kgrid = collect(0.5*k_ss:step:1.5*k_ss) # G_{k_{t+1}} in the paper
    #kgrid = collect(linspace(0.5*k_ss,1.5*k_ss,10))


    nk = length(kgrid)
    nz = length(zgrid)
    
 
    # Generate grid of market resorces
    cih =  (kgrid.^α)*zgrid #(1.0-δ)*kgrid needed for the basic setting.

    # Define utility function
    u(c) = log(c)
    u′(c) = 1/c
    u′_inv(val) = 1/val

    ############################################################################
    ## 1. Guess Ṽ0(k_{t+1},z_t) = βEV(k_{t+1},z_{t+1})
    ############################################################################


    ṽ0 = zeros(nk,nz)
    #ṽ0 has to be increasing in capital
    value_init = u(c_ss)
    for i in 1:nk
        @views @inbounds ṽ0[i,:] = (1.0/(1.0-β))*value_init + i/nk
    end
    
    # Initialization
    ṽ1 = similar(ṽ0)
    D_ṽ0 = similar(ṽ0)
    c = similar(ṽ0)
    yend = similar(ṽ0)
    V0 = similar(ṽ0)
    V1 = similar(ṽ0)


    # Iteration settings
    const maxiter = 10000
    const tol = 1e-7

    #### Start iteration
    for iter in 1: maxiter
        ############################################################################
        ## 2. Compute the derivative to obtain ṽ_k0,optimal consumption
        ###########################################################################

        @views @inbounds D_ṽ0[1,:] = (ṽ0[2,:] - ṽ0[1,:])/step
        @views @inbounds D_ṽ0[end,:] = (ṽ0[end,:]-ṽ0[end-1,:])/step
        for i in 2:nk-1
            @views @inbounds D_ṽ0[i,:] =(ṽ0[i+1,:]-ṽ0[i-1,:])/(2.0*step)
        end
        
        #println("Derivetive is $D_ṽ0")
        c = u′_inv.(D_ṽ0)
        #println("optimal consumption is $c")

        ########################################################################
        ## 3. Update the value function.
        ##    Compute endogenously determined market resorces as yend(kend_t,z_t)
        ##    kend is k_t that is endogenously determined
        ########################################################################
        V0 = u.(c) + ṽ0 #普通のv 

        # need to evaluate Value as a function of cash in hand but not yend

        for z in 1:nz
            @views @inbounds yend[:,z] = c[:,z] + kgrid
            @views @inbounds V1[:,z] = interp1(yend[:,z],V0[:,z],cih[:,z], method="linear")
        end


        ########################################################################
        # 4. Compute ṽ^{n+1} = βEV^{n+1}(Y_{t+1},z_{t+1})
        ########################################################################
        ṽ1 = β*(V1*Π')

        ########################################################################
        # 5. Compute error
        ########################################################################
        err = maximum(abs.(ṽ1-ṽ0))
        ṽ0 = copy(ṽ1)
        if err < tol
            break
        end

    end
    #### End of iteration.
    println("Loop is Finished")
    ############################################################################
    # 6. Use a nonlinear solver to get kend_t(z_t)
    #    RECALL! kend is k_t that is endogenously determined
    ############################################################################
    # To get kend_t, we need to define function like resorce constraint
    kend = copy(yend)
    println("yend = $yend")
    for i in 1:nk
        #test = kgrid[i]
        for z in 1:nz
        #    @inbounds kend[i,z] = mynewton(yend[i,z],zgrid[z],α,test)
            @inbounds egm_f(x) = yend[i,z] - zgrid[z]*x^α
            @inbounds kend[i,z] = fzero(egm_f,kgrid[i])
        end

    end

    # compare the results to the standard algorithm, we need interpolation.
    value =similar(kend)
    grid_k = similar(kend) # policy function for capital
    grid_c= similar(kend) # policy function for consumption

    for z in 1:nz
        @views @inbounds value[:,z] = interp1(kend[:,z],V0[:,i],kgrid,method="linear")
        @views @inbounds grid_k = interp1(kend[:,z],kgrid,kgrid,method="linear")
        @views @inbounds grid_c[:,z] = interp1(kend[:,z],c[:,z],kgrid,method="linear" )
    end
    println("computation is finished")
    return value, grid_k, grid_c,kgrid
end


main (generic function with 1 method)

In [5]:
v, pol_k,pol_c,kgrid = main()

Output = 0.5627314338711378, Capital = 0.178198287392527, Consumption = 0.3845331464786108
Loop is Finished


Excessive output truncated after 809903 bytes.

LoadError: DomainError:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.